# Analyse et visualisation de données avec Python
## Combiner des DataFrames avec Pandas
Questions
* Peut-on travaillers avec plusieurs sources de données?
* Comment combiner les données de deux DataFrames?

Objectifs
* Combiner les données de plusieurs fichiers en utilisant `concat` et `merge`.
* Combiner deux DataFrames utilisant un identifiant commun.

## Charger nos données

In [ ]:
# Charger le module pandas
import pandas as pd

# Lister une collection de fichiers CSV
from glob import glob
glob('../data/by_year/*.csv')

## Concaténer des DataFrames

In [ ]:
annee2001 = pd.read_csv('../data/by_year/surveys_2001.csv')
annee2002 = pd.read_csv('../data/by_year/surveys_2002.csv')

print(annee2001.shape, annee2002.shape)

In [ ]:
# Concaténer les dataframes verticalement
vertical = pd.concat([annee2001, annee2002], axis='index')
vertical

In [ ]:
# Réinitaliser l'index du dataframe
# L'option drop=True évite l'ajout d'une colonne avec l'ancien index
vertical = vertical.reset_index(drop=True)
vertical

In [ ]:
# Accumuler les données de tous les fichiers de la collection
surveys_df = pd.DataFrame()  # DataFrame vide

for fichier in glob('../data/by_year/*.csv'):
    df_annee = pd.read_csv(fichier)
    surveys_df = pd.concat([surveys_df, df_annee], axis='index')

surveys_df = surveys_df.reset_index(drop=True)
surveys_df

## Exercice - Concaténer des DataFrames
* Dans `surveys_df`, sélectionnez individuellement les enregistrements des années 2001 et 2002
* Concaténez les deux dataframes verticalement

(3 min.)

In [ ]:
# Obtenir les données pour chaque année
annee2001 = surveys_df[surveys_df['year'] == 2001]
annee2002 = surveys_df[surveys_df['year'] == 2002]

# Concaténer verticalement
deux_annees = pd.concat([annee2001, annee2002], axis='index')

* Calculez le poids moyen selon l'année et le sexe (1 min.)

In [ ]:
# Calculer le poids moyen par année et par sexe
poids_annee = deux_annees.groupby(['year', 'sex'])['weight'].mean()
poids_annee = poids_annee.unstack()
poids_annee

* Sauvegardez le tableau des moyennes
  dans un fichier CSV et le recharger (2 min.)

In [ ]:
# Écrire dans un fichier - garder l'index 'year' cette fois-ci
fichier_csv = 'poids_par_annee.csv'
poids_annee.to_csv(fichier_csv, index=True)

# Relire les données, fournir le nom de l'index
pd.read_csv(fichier_csv, index_col='year')

## Joindre deux DataFrames

In [ ]:
# Importer un sous-ensemble des espèces pour cet exemple
trois_especes = pd.read_csv('../data/speciesSubset.csv')
trois_especes

### Identifier les clés de jonction

In [ ]:
premiers10.columns

In [ ]:
trois_especes.columns

### Une intersection ou "inner join"

![Inner join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/inner-join.png)

In [ ]:
# Calculer l'intersection de premiers10 et trois_especes
cle = 'species_id'
intersection = pd.merge(left=premiers10, right=trois_especes,
                        left_on=cle, right_on=cle)
# Quelle est la taille de la jonction?
intersection.shape

In [ ]:
intersection

### Jonction de gauche

![Left join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/left-join.png)

In [ ]:
jonc_gauche = pd.merge(left=premiers10, right=trois_especes,
                       on=cle, how='left')
# Quelle est la taille de la jonction?
jonc_gauche.shape

In [ ]:
jonc_gauche

### Les autres types de jonction
* `how='right'` : toutes les lignes du second DataFrame sont gardées
* `how='outer'` : équivalent d'une union, toutes les lignes sont gardées

## Exercice - Joindre toutes les données
`1`. Créez un nouveau DataFrame tel que tous les
enregistrements de `surveys.csv` sont gardés dans une jonction
impliquant les informations correspondantes de `species.csv`.
(3 min.)

In [ ]:
species_df = pd.read_csv('../data/species.csv')

jonc_gauche = pd.merge(
    left=surveys_df, right=species_df, on='species_id', how='left')
jonc_gauche.shape

`2`. Calculez et créez un graphique (*bar-plot*) montrant
le nombre d'enregistrements (soit le nombre de `record_id`)
par type de `taxa` pour chaque site (`plot_id`). (3 min.)

In [ ]:
par_site_taxa = jonc_gauche.groupby(['plot_id', 'taxa'])
nb_par_site_taxa = par_site_taxa['record_id'].count().unstack()
nb_par_site_taxa.tail()

In [ ]:
nb_par_site_taxa.plot(kind='bar', logy=True)

`3`. Calculez et créez un graphique (bar-plot) montrant le nombre
de différents `taxa` par type de sexe pour chaque site (`plot_id`).
(2 min.)

In [ ]:
# Nettoyage des données
jonc_gauche['sex'] = jonc_gauche['sex'].fillna('F|M')
invalides = ~jonc_gauche['sex'].isin(['F', 'F|M', 'M'])
jonc_gauche.loc[invalides, 'sex'] = 'F|M'

In [ ]:
ntaxa_sex_site = jonc_gauche.groupby(
    ['plot_id', 'sex'])['taxa'].nunique().reset_index(level=1)
ntaxa_sex_site.tail()

In [ ]:
# Utiliser pivot_table() au lieu de unstack()
pivot_taxa_sex_site = ntaxa_sex_site.pivot_table(
    values='taxa', columns='sex', index=ntaxa_sex_site.index)
pivot_taxa_sex_site.tail()

In [ ]:
pivot_taxa_sex_site.plot(kind="bar")

## Résumé technique
* **Concaténer** des DataFrames avec `pandas.concat()`
  * Requiert une liste de DataFrames
  * Verticalement si `axis='index'` (par défaut)
  * Horizontalement si `axis='columns'`
  * Réinitialiser l'index au besoin : `reset_index(drop=True)`
* **Joindre** des DataFrames avec `pandas.merge()`
  * `left=`, `right=` : les deux DataFrames à joindre
  * `left_on=`, `right_on=` : les clés de jonction de chaque DataFrame
  * `on=` : clés de jonction communes aux deux DataFrames
  * `how=` : `'inner'` (défaut), `'left'`, `'right'`, `'outer'`
* **Table de pivot** : `pivot_table()`
  * `values=colX`
  * `index=[col_ind]`
  * `columns=[categorie1, categorie2]`
  * `aggfunc=numpy.mean` (défaut: moyenne)